# Spark SQL and DataFrames

### 1. Basic manipulations
First, let's create some users:

In [0]:
from datetime import date

# Let's first create some users:
col_names = ["first_name", "last_name", "birth_date", "gender", "country"]
users = [
  ("Alice", "Jones", date(1981, 4, 15), "female", "Canada"),
  ("John", "Doe", date(1951, 1, 21), "male", "USA"),
  ("Barbara", "May", date(1951, 9, 1), "female", "Australia"),
  ("James", "Smith", date(1975, 7, 12), "male", "United Kingdom"),
  ("Gerrard", "Dupont", date(1968, 5, 9), "male", "France"),
  ("Amanda", "B.", date(1988, 12, 16), "female", "New Zeland")
]

users_df = spark.createDataFrame(users, col_names)
display(users_df) # Only works in Databricks. Elswehere, use "df.show()" or "df.toPandas()"

first_name,last_name,birth_date,gender,country
Alice,Jones,1981-04-15,female,Canada
John,Doe,1951-01-21,male,USA
Barbara,May,1951-09-01,female,Australia
James,Smith,1975-07-12,male,United Kingdom
Gerrard,Dupont,1968-05-09,male,France
Amanda,B.,1988-12-16,female,New Zeland


Now it's your turn, create more users (at least 3, with different names) and add to the initial users, saving the result in a new variable.

In [0]:
new_users_df = [
    ("Charlotte", "Imbert", date(1998, 4, 30), "female", "France"),
    ("Yann", "Bredy-Maux", date(1986, 3, 12), "male", "Egypt"),
    ("Amine", "Lpb", date(1993, 5, 13), "male", "China")
]
 
new_users_df = spark.createDataFrame(new_users_df, col_names)
all_users_df = users_df.unionByName(new_users_df).dropDuplicates()
display(all_users_df) # or all_users_df.show()

first_name,last_name,birth_date,gender,country
Alice,Jones,1981-04-15,female,Canada
John,Doe,1951-01-21,male,USA
Barbara,May,1951-09-01,female,Australia
James,Smith,1975-07-12,male,United Kingdom
Gerrard,Dupont,1968-05-09,male,France
Amanda,B.,1988-12-16,female,New Zeland
Charlotte,Imbert,1998-04-30,female,France
Yann,Bredy-Maux,1986-03-12,male,Egypt
Amine,Lpb,1993-05-13,male,China


Now, select only two columns and show the resulting DataFrame, without saving it into a variable.

In [0]:
all_users_df.select("first_name", "last_name").show()

+----------+----------+
|first_name| last_name|
+----------+----------+
|     Alice|     Jones|
|      John|       Doe|
|   Barbara|       May|
|     James|     Smith|
|   Gerrard|    Dupont|
|    Amanda|        B.|
| Charlotte|    Imbert|
|      Yann|Bredy-Maux|
|     Amine|       Lpb|
+----------+----------+



Now, register your DataFrame as a table and select the same two columns with a SQL query string

In [0]:
query_string = "SELECT first_name, last_name FROM df_final_View"
all_users_df.createOrReplaceTempView('df_final_View') #source_2 # creates a local temporary table accessible by a SQL query
spark.sql(query_string).show()

+----------+----------+
|first_name| last_name|
+----------+----------+
|     Alice|     Jones|
|      John|       Doe|
|   Barbara|       May|
|     James|     Smith|
|   Gerrard|    Dupont|
|    Amanda|        B.|
| Charlotte|    Imbert|
|      Yann|Bredy-Maux|
|     Amine|       Lpb|
+----------+----------+



Now we want to add an unique identifier for each user in the table. There are many strategies for that, and for our example we will use the string `{last_name}_{first_name}`

You can use `all_users_df` since your latest operation did not override its values. Add a new column called `user_id` to your DataFrame and save to a new variable.

**Hint:** The first place to look is in the [functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions) package

In [0]:
from pyspark.sql import functions as fn
from pyspark.sql.functions import monotonically_increasing_id
 
 
users_with_id = all_users_df.withColumn("user_id", monotonically_increasing_id())
display(users_with_id)

first_name,last_name,birth_date,gender,country,user_id
Alice,Jones,1981-04-15,female,Canada,0
John,Doe,1951-01-21,male,USA,1
Barbara,May,1951-09-01,female,Australia,2
James,Smith,1975-07-12,male,United Kingdom,3
Gerrard,Dupont,1968-05-09,male,France,4
Amanda,B.,1988-12-16,female,New Zeland,5
Charlotte,Imbert,1998-04-30,female,France,6
Yann,Bredy-Maux,1986-03-12,male,Egypt,7
Amine,Lpb,1993-05-13,male,China,8


You can also do the same thing with an User Defined Function by passing a lambda, although it is not recommended when there is already a function in the `functions` package.

Add a new column called `user_id_udf` to the DataFrame, using an UDF that receives two parameters and concatenate them.

In [0]:
concat_udf = fn.udf(lambda first, last: '_'.join([last, first]))
users_with_id = all_users_df.withColumn("user_id", concat_udf('last_name', 'first_name'))
display(users_with_id)

first_name,last_name,birth_date,gender,country,user_id
Alice,Jones,1981-04-15,female,Canada,Alice_Jones
John,Doe,1951-01-21,male,USA,John_Doe
Barbara,May,1951-09-01,female,Australia,Barbara_May
James,Smith,1975-07-12,male,United Kingdom,James_Smith
Gerrard,Dupont,1968-05-09,male,France,Gerrard_Dupont
Amanda,B.,1988-12-16,female,New Zeland,Amanda_B.
Charlotte,Imbert,1998-04-30,female,France,Charlotte_Imbert
Yann,Bredy-Maux,1986-03-12,male,Egypt,Yann_Bredy-Maux
Amine,Lpb,1993-05-13,male,China,Amine_Lpb


Now, let's add another column called `age` with the computed age (in years) of each user, based on a given reference date, and save the resulting DataFrame into a new variable.

**Hint:** You can first compute the age in months, and then divide by 12. A final operation will probably be needed to get an integer number.

In [0]:
users_from_file = users_with_id.withColumn(
    'age',
    fn.floor(fn.months_between(fn.lit(date(2022, 1, 24)), 'birth_date') / 12)
)
users_from_file.show()

+----------+----------+----------+------+--------------+----------------+---+
|first_name| last_name|birth_date|gender|       country|         user_id|age|
+----------+----------+----------+------+--------------+----------------+---+
|     Alice|     Jones|1981-04-15|female|        Canada|     Alice_Jones| 40|
|      John|       Doe|1951-01-21|  male|           USA|        John_Doe| 71|
|   Barbara|       May|1951-09-01|female|     Australia|     Barbara_May| 70|
|     James|     Smith|1975-07-12|  male|United Kingdom|     James_Smith| 46|
|   Gerrard|    Dupont|1968-05-09|  male|        France|  Gerrard_Dupont| 53|
|    Amanda|        B.|1988-12-16|female|    New Zeland|       Amanda_B.| 33|
| Charlotte|    Imbert|1998-04-30|female|        France|Charlotte_Imbert| 23|
|      Yann|Bredy-Maux|1986-03-12|  male|         Egypt| Yann_Bredy-Maux| 35|
|     Amine|       Lpb|1993-05-13|  male|         China|       Amine_Lpb| 28|
+----------+----------+----------+------+--------------+--------

Now, an analytical question: How many users of each gender who are more than 40 years old exist in this data? The solution must be a DataFrame with two columns: `age` and `count` and two lines, one for each gender.

Bonus: Try to do your solution in a single chain (without intermediate variables)

**Hint:** You will need to filter and aggregate the data.

In [0]:
users_from_file.filter('age > 40').groupBy("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|female|    1|
|  male|    3|
+------+-----+



### 2. Reading files, performing joins, and aggregating

For this section you will use some fake data of two datasets: `Users` and `Donations`. The data is provided in two CSV files *with header* and using *comma* as separator.

The `Users` dataset contains information about about the users. 

The `Donations` dataset contains information about Donations performed by those users.

The first task is to read these files into the appropriate DataFrames.

**Note:** You need to set the option "inferSchema" to true in order to have the columns in the correct types.

_The data for this section has been created using [Mockaroo](https://www.mockaroo.com/)_.

In [0]:
users_from_file = spark\
    .read\
    .option("inferSchema", "True")\
    .option("header", "true")\
    .format("csv")\
    .load("/FileStore/tables/users.csv")
    
donations = spark\
    .read\
    .option("inferSchema", "True")\
    .option("header", "true")\
    .format("csv")\
    .load("/FileStore/tables/donations.csv")

Now investigate the columns, contents and size of both datasets

In [0]:
# print the column names and types
users_from_file.printSchema()
donations.printSchema()
 
# print 5 elements of the datasets in a tabular format
users_from_file.show(n=5)
donations.show(n=5)
 
# print the number of lines of each dataset
print(users_from_file.count())
print(donations.count())

root
 |-- user_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- country: string (nullable = true)

root
 |-- donation_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- donation_value: double (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- timestamp: string (nullable = true)

+-------+----------+---------+------+----------+-------------+
|user_id|first_name|last_name|gender|birth_date|      country|
+-------+----------+---------+------+----------+-------------+
|      1|   Janifer|   Lagadu|Female|1940-08-09|United States|
|      2|     Bruis|  Danilov|  Male|1966-05-28|       Brazil|
|      3|      Omar| Spinozzi|  Male|1977-08-28|        China|
|      4|       Mag|  Kennedy|Female|1963-11-11|        China|
|      5|    Gerome|   Lushey|  Male|1990-06-08|       Brazil|

**Note:** If all the column types shown in the previous results are "string", you need to make sure you passed "inferSchema" as true when reading the CSV files before continuing.

Before using the data, we may want to add some information about the users. 

Add a column containing the age of each user.

In [0]:
users_from_file = users_from_file.withColumn(
    'age',
    fn.floor(
        fn.months_between(
            fn.lit(date(2022, 1, 29)),
            'birth_date'
        ) / 12
))

users_from_file.show(n=5)

+-------+----------+---------+------+----------+-------------+---+
|user_id|first_name|last_name|gender|birth_date|      country|age|
+-------+----------+---------+------+----------+-------------+---+
|      1|   Janifer|   Lagadu|Female|1940-08-09|United States| 81|
|      2|     Bruis|  Danilov|  Male|1966-05-28|       Brazil| 55|
|      3|      Omar| Spinozzi|  Male|1977-08-28|        China| 44|
|      4|       Mag|  Kennedy|Female|1963-11-11|        China| 58|
|      5|    Gerome|   Lushey|  Male|1990-06-08|       Brazil| 31|
+-------+----------+---------+------+----------+-------------+---+
only showing top 5 rows



Another useful information to have is the age **range** of each user. Using the `when` function, create the following 5 age ranges:
- "(0, 25]"
- "(25, 35]"
- "(35, 45]"
- "(45, 55]"
- "(55, ~]"

And add a new column to the users DataFrame, containing this information.

**Note:** When building logical operations with Spark DataFrames, it's better to be add parantheses. Example:
```python
df.select("name", "age").where( (df("age") > 20) & (df("age") <= 30) )
```

**Note 2:** If you are having problems with the `when` function, you can make an User Defined Function and do your logic in standard python.

In [0]:
# Cast Integer type for the new Age column
users_from_file = users_from_file.withColumn(
    "age",
    users_from_file.age.cast('int')
)

users_from_file.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- country: string (nullable = true)
 |-- age: integer (nullable = true)



In [0]:
users_from_file = users_from_file.withColumn(
    "age_range",
    fn.when(
        (fn.col("age") <= 25), "(0, 25]"
    ).when(
        (fn.col("age") > 25) & (fn.col("age") <= 35), "(25, 35]"
    ).when(
        (fn.col("age") > 35) & (fn.col("age") <= 45), "(35, 45]"
    ).when(
        (fn.col("age") > 45) & (fn.col("age") <= 55), "(45, 55]"
    ).otherwise("(55, ~]")
)

users_from_file.show(n=5)

+-------+----------+---------+------+----------+-------------+---+---------+
|user_id|first_name|last_name|gender|birth_date|      country|age|age_range|
+-------+----------+---------+------+----------+-------------+---+---------+
|      1|   Janifer|   Lagadu|Female|1940-08-09|United States| 81|  (55, ~]|
|      2|     Bruis|  Danilov|  Male|1966-05-28|       Brazil| 55| (45, 55]|
|      3|      Omar| Spinozzi|  Male|1977-08-28|        China| 44| (35, 45]|
|      4|       Mag|  Kennedy|Female|1963-11-11|        China| 58|  (55, ~]|
|      5|    Gerome|   Lushey|  Male|1990-06-08|       Brazil| 31| (25, 35]|
+-------+----------+---------+------+----------+-------------+---+---------+
only showing top 5 rows



Now that we have improved our users' DataFrame, the first analysis we want to make is the average donation performed by each gender. However, the gender information and the donation value are in different tables, so first we need to join them, using the `user_id` as joining key.

**Note:** Make sure you are not using the `users` DataFrame from the first part of the lab.

**Note 2:** For better performance, you can [broadcast](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.functions.broadcast.html#pyspark.sql.functions.broadcast) the smaller dataset. But you should only do this when you are sure the DataFrame fits in the memory of the nodes.

In [0]:
joined_df = users_from_file.join(
    donations,
    users_from_file.user_id == donations.user_id,
    "inner"
).drop(users_from_file.user_id)

fn.broadcast(joined_df)

Out[14]: DataFrame[first_name: string, last_name: string, gender: string, birth_date: string, country: string, age: int, age_range: string, donation_id: int, user_id: int, donation_value: double, date: string, time: string, timestamp: string]

In [0]:
display(joined_df)

first_name,last_name,gender,birth_date,country,age,age_range,donation_id,user_id,donation_value,date,time,timestamp
Tabor,Bramer,Male,1983-05-29,Brazil,38,"(35, 45]",1,63,25.41,2017-03-06,18:28:20,2017-03-06 18:28:20
Chris,Climie,Male,1970-06-02,France,51,"(45, 55]",2,53,93.32,2017-03-15,15:15:09,2017-03-15 15:15:09
Livia,Lodemann,Female,1967-03-11,United States,54,"(45, 55]",3,74,136.04,2017-10-29,20:36:23,2017-10-29 20:36:23
Kendre,Pankhurst.,Female,1941-05-28,China,80,"(55, ~]",4,94,37.25,2017-05-16,12:33:58,2017-05-16 12:33:58
Quincy,Marquess,Male,1997-08-13,Brazil,24,"(0, 25]",5,83,89.45,2017-05-09,22:45:44,2017-05-09 22:45:44
Boothe,Endley,Male,1960-03-03,China,61,"(55, ~]",6,80,163.67,2017-07-25,21:50:36,2017-07-25 21:50:36
Sharl,Airy,Female,1941-09-08,Brazil,80,"(55, ~]",7,19,10.69,2017-11-15,8:43:54,2017-11-15 8:43:54
Chris,Climie,Male,1970-06-02,France,51,"(45, 55]",8,53,135.09,2017-04-14,7:43:37,2017-04-14 7:43:37
Ethelyn,Raubenheim,Female,1953-07-19,Brazil,68,"(55, ~]",9,46,94.38,2017-03-07,10:30:34,2017-03-07 10:30:34
Farlee,O'Connell,Male,1985-05-15,China,36,"(35, 45]",10,90,68.63,2017-02-02,14:23:43,2017-02-02 14:23:43


Now, use aggregation to find the the min, max and avg donation by gender.

In [0]:
donations_by_gender = joined_df.groupby("gender").agg(
    fn.min("donation_value").alias("min_donation"),\
    fn.max("donation_value").alias("max_donation"),\
    fn.mean("donation_value").alias("avg_donation")
)
donations_by_gender.show()

+------+------------+------------+------------------+
|gender|min_donation|max_donation|      avg_donation|
+------+------------+------------+------------------+
|Female|        1.25|      199.87| 97.62667400881057|
|  Male|        1.12|      199.93|100.23606227106244|
+------+------------+------------+------------------+



Now, make the necessary transformations and aggregations to answer to the following questions about the data. Note that some questions are only about the users, so make sure to use the smaller possible dataset when looking for your answers!

**Question 1:** 

a) What's the average, min and max age of the users? 

b) What's the average, min and max age of the users, by gender?

In [0]:
result_1a = users_from_file.describe("age")
result_1a.show()

result_1b = users_from_file.groupby("gender").agg(
    fn.min("age").alias("min_age"),\
    fn.max("age").alias("max_age"),\
    fn.mean("age").alias("avg_age")
)
result_1b.show()

+-------+-----------------+
|summary|              age|
+-------+-----------------+
|  count|              100|
|   mean|            52.26|
| stddev|16.36738565352797|
|    min|               23|
|    max|               81|
+-------+-----------------+

+------+-------+-------+-----------------+
|gender|min_age|max_age|          avg_age|
+------+-------+-------+-----------------+
|Female|     23|     81|56.61224489795919|
|  Male|     23|     79|48.07843137254902|
+------+-------+-------+-----------------+



**Question 2:**

a) How many distinct country origins exist in the data? Print a DataFrame listing them.

b) What are the top 5 countries with the most users? Print a DataFrame containing the name of the countries and the counts.

In [0]:
result_2a = users_from_file.select("country").distinct()
result_2a.show()

result_2b = users_from_file.groupBy("country")\
    .count().sort("count", ascending=False)
result_2b.show(n=5)

+-------------+
|      country|
+-------------+
|      Germany|
|       France|
|United States|
|        China|
|      Nigeria|
|       Brazil|
|        Japan|
|  New Zealand|
+-------------+

+-------------+-----+
|      country|count|
+-------------+-----+
|        China|   54|
|       Brazil|   19|
|       France|   13|
|United States|    5|
|        Japan|    4|
+-------------+-----+
only showing top 5 rows



**Question 3:**

What's the number of donations average, min and max donations values by age range?

In [0]:
result_3 = joined_df.groupBy("age_range").agg(
    fn.min("donation_value").alias("min_donation"),\
    fn.max("donation_value").alias("max_donation"),\
    fn.mean("donation_value").alias("avg_donation")
)
result_3.show()

+---------+------------+------------+------------------+
|age_range|min_donation|max_donation|      avg_donation|
+---------+------------+------------+------------------+
| (25, 35]|        1.12|      199.28| 91.04742857142855|
|  (55, ~]|        1.19|      199.91|  99.4394789081886|
| (45, 55]|        2.49|      199.93|102.55648780487805|
|  (0, 25]|        4.33|      198.15|101.61066666666667|
| (35, 45]|        2.03|      199.87| 99.68159420289848|
+---------+------------+------------+------------------+



**Question 4:**

a) What's the number of donations, average, min and max donation values by user location (country)?

b) What is the number of donations, average, min and max donation values by gender for each user location (contry)? (the resulting DataFrame must contain 5 columns: the gender of the user, their country and the 3 metrics)

In [0]:
result_4a = joined_df.groupBy("country").agg(
    fn.min("donation_value").alias("min_donation"),\
    fn.max("donation_value").alias("max_donation"),\
    fn.mean("donation_value").alias("avg_donation")
)
result_4a.show()

result_4b = joined_df.groupBy("country", "gender").agg(
    fn.min("donation_value").alias("min_donation"),\
    fn.max("donation_value").alias("max_donation"),\
    fn.mean("donation_value").alias("avg_donation")
)
result_4b.show()

+-------------+------------+------------+------------------+
|      country|min_donation|max_donation|      avg_donation|
+-------------+------------+------------+------------------+
|      Germany|       17.81|      180.48| 95.64157894736842|
|       France|        5.79|      199.93| 95.63199999999999|
|United States|        1.36|      198.15|          108.3796|
|        China|        1.25|      199.32| 97.62353463587931|
|      Nigeria|       14.47|      192.63| 81.22600000000001|
|       Brazil|        1.12|      199.91|105.70655555555568|
|        Japan|        1.19|      199.76| 95.63270270270273|
|  New Zealand|       12.98|      172.51|            107.93|
+-------------+------------+------------+------------------+

+-------------+------+------------+------------+------------------+
|      country|gender|min_donation|max_donation|      avg_donation|
+-------------+------+------------+------------+------------------+
|        Japan|  Male|        1.19|      199.76| 86.94461538461

**Question 5**

Which month of the year has the largest aggregated donation value?

**Hint**: you can use a function to extract the month from a date, then you can aggregate to find the total donation value.

In [0]:
# Add month column
joined_df = joined_df.withColumn("month", fn.month("date"))

# Group by month
result_5 = joined_df.groupBy("month")\
    .agg({"donation_value" : "sum"})\
    .sort("sum(donation_value)", ascending=False)

result_5.show(n=1)

+-----+-------------------+
|month|sum(donation_value)|
+-----+-------------------+
|    5| 10696.400000000001|
+-----+-------------------+
only showing top 1 row



### 3. Window Functions

_This section uses the same data as the last one._

Window functions are very useful for gathering aggregated data without actually aggregating the DataFrame. They can also be used to find "previous" and "next" information for entities, such as an user. [This article](https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html) has a very nice explanation about the concept.

We want to find the users who donated less than a threshold and remove them from the donations dataset. Now, there are two ways of doing that:

1) Performing a traditional aggregation to find the users who donated less than the threshold, then filtering these users from the donations dataset, either with `where(not(df("user_id").isin(a_local_list)))` or with a join of type "anti-join".

2) Using window functions to add a new column with the aggregated donations per user, and then using a normal filter such as `where(aggregated_donation < threshold)`

Let's implement both and compare the complexity:

First, perform the traditional aggregation and find the users who donated less than 500 in total:

In [0]:
bad_users = joined_df.groupBy("user_id")\
    .agg({"donation_value": "sum"})\
    .filter(fn.col("sum(donation_value)") <= 500)
bad_users.show()

+-------+-------------------+
|user_id|sum(donation_value)|
+-------+-------------------+
|     85| 437.78999999999996|
|     47|             351.84|
|     96|             354.97|
|     84| 329.46999999999997|
|     87| 497.60999999999996|
|     82|             291.73|
|     73| 455.20000000000005|
|     25| 413.09000000000003|
|     42|             215.35|
|     99|             370.99|
+-------+-------------------+



You should have found around 10 users. Now, perform an "anti-join" to remove those users from `joined_df`.

**Hint:** The `join` operation accepts a third argument which is the join type. The accepted values are: 'inner', 'outer', 'full', 'fullouter', 'leftouter', 'left', 'rightouter', 'right', 'leftsemi', 'leftanti', 'cross'.

In [0]:
good_donations = joined_df.join(
    bad_users,\
    joined_df.user_id == bad_users.user_id,\
    "leftanti"
)
good_donations.count()

Out[38]: 955

Verify if the count of `good_donations` makes sense by performing a normal join to find the `bad_donations`.

In [0]:
bad_donations = joined_df.join(
    bad_users,\
    joined_df.user_id == bad_users.user_id,\
    "inner"
)
bad_donations.count()

Out[26]: 45

If you done everything right, at this point `good_donations.count()` + `bad_donations.count()` = `joined_df.count()`.

But using the join approach can be very heavy and it requires multipe operations. For this kind of problems, Window Functions are better.

In [0]:
good_donations.count() + bad_donations.count() == joined_df.count()

Out[39]: True

The first step is to create your window specification over `user_id` by using partitionBy

In [0]:
from pyspark.sql import Window

window_spec = Window.partitionBy("user_id")

Then, you can use one of the window functions of the `pyspark.sql.functions` package, appling to the created window_spec by using the `over` method. 

**Hint:** If you are blocked, try looking at the [documentation](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.Column.over.html#pyspark.sql.Column.over) for the `over` method, or searching on StackOverflow.

In [0]:
new_column = fn.sum("donation_value").over(window_spec)

donations_with_total = joined_df.withColumn("total_donations", new_column)
donations_with_total.show()

+----------+---------+------+----------+-------------+---+---------+-----------+-------+--------------+----------+--------+-------------------+-----+------------------+
|first_name|last_name|gender|birth_date|      country|age|age_range|donation_id|user_id|donation_value|      date|    time|          timestamp|month|   total_donations|
+----------+---------+------+----------+-------------+---+---------+-----------+-------+--------------+----------+--------+-------------------+-----+------------------+
|   Janifer|   Lagadu|Female|1940-08-09|United States| 81|  (55, ~]|        145|      1|         95.88|2017-05-13| 5:27:25| 2017-05-13 5:27:25|    5|            889.16|
|   Janifer|   Lagadu|Female|1940-08-09|United States| 81|  (55, ~]|        264|      1|        150.51|2017-09-21| 4:36:37| 2017-09-21 4:36:37|    9|            889.16|
|   Janifer|   Lagadu|Female|1940-08-09|United States| 81|  (55, ~]|        317|      1|         51.55|2017-10-26| 1:55:37| 2017-10-26 1:55:37|   10|      

And now you can just filter on the `total_donated_by_user` column:

In [0]:
good_donations_wf = donations_with_total.filter(fn.col("total_donations") > 500)
good_donations_wf.count()

Out[46]: 955

If you done everything right, you should obtain `good_donations_wf.count()` = `good_donations.count()`

In [0]:
good_donations_wf.count() == good_donations.count()

Out[47]: True

Window functions also can be useful to find the "next" and "previous" operations by using `functions.lead` and `functions.lag`. In our example, we can use it to find the date interval between two donations by a specific user.

For this kind of Window function, the window specification must be ordered by the date. So, create a new window specification partitioned by the `user_id` and ordered by the donation timestamp.

In [0]:
ordered_window = Window.partitionBy("user_id").orderBy("timestamp")

Now use `functions.lag().over()` to add a column with the timestamp of the previous donation of the same user. Then, inspect the result to see what it looks like.

In [0]:
new_column = fn.lag("timestamp").over(ordered_window)

donations_with_lag = good_donations.withColumn("previous_timestamp", new_column)
donations_with_lag.orderBy("user_id", "timestamp").show()

+----------+---------+------+----------+-------------+---+---------+-----------+-------+--------------+----------+--------+-------------------+-----+-------------------+
|first_name|last_name|gender|birth_date|      country|age|age_range|donation_id|user_id|donation_value|      date|    time|          timestamp|month| previous_timestamp|
+----------+---------+------+----------+-------------+---+---------+-----------+-------+--------------+----------+--------+-------------------+-----+-------------------+
|   Janifer|   Lagadu|Female|1940-08-09|United States| 81|  (55, ~]|        560|      1|        121.82|2016-12-03| 3:34:03| 2016-12-03 3:34:03|   12|               null|
|   Janifer|   Lagadu|Female|1940-08-09|United States| 81|  (55, ~]|        486|      1|        188.37|2017-02-01|20:20:44|2017-02-01 20:20:44|    2| 2016-12-03 3:34:03|
|   Janifer|   Lagadu|Female|1940-08-09|United States| 81|  (55, ~]|        145|      1|         95.88|2017-05-13| 5:27:25| 2017-05-13 5:27:25|    5|2

Finally, compute the average time it took for each user between two of their consecutive donations (in days), and print the 5 users with the smallest averages. The result must include at least the users' id, last name and birth date, as well as the computed average.

In [0]:
donations_with_days_diff = donations_with_lag.withColumn(\
    "days_between_donations",\
    fn.datediff(\
        fn.col("timestamp"), fn.col("previous_timestamp")\
))

donations_with_avg_days_diff = donations_with_days_diff.groupBy("user_id", "last_name", "birth_date")\
    .agg({"days_between_donations": "avg"})\
    .sort("avg(days_between_donations)")

donations_with_avg_days_diff.show(n=5)

+-------+---------+----------+---------------------------+
|user_id|last_name|birth_date|avg(days_between_donations)|
+-------+---------+----------+---------------------------+
|     67| Hambatch|1954-04-14|          14.23076923076923|
|     78| Sherston|1965-07-23|                       16.0|
|     91|  Kildale|1979-01-25|          17.88888888888889|
|      6|   Andrus|1983-04-22|         18.529411764705884|
|     63|   Bramer|1983-05-29|          18.88888888888889|
+-------+---------+----------+---------------------------+
only showing top 5 rows



Congratulations, you have finished this notebook!